In [1]:
import requests
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from tqdm.notebook import trange, tqdm
from time import sleep

Address coordinates

In [3]:
## Function for getting postal code, geo coordinates of addresses

def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(tqdm(lst)):
        # Do not need to change the URL
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        # print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        try:
            data = json.loads(response.text) 
        except ValueError:
            print('JSONDecodeError')
            pass
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    
    file.to_csv(filename + '.csv')

In [4]:
## Function for getting closest distance of each location from a list of amenities location

from geopy.distance import geodesic

def find_nearest(house, amenity, radius=2):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    Both are dataframes with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    It also finds the number of amenities within the given radius (default=2)
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(tqdm(house.iloc[:,0])):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100,0]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2])
            distance = geodesic(flat_loc,amenity_loc)
            distance = float(str(distance)[:-3]) # convert to float

            if distance <= radius:   # compute number of amenities in 2km radius
                flat_amenity[3] += 1

            if distance < flat_amenity[2]: # find nearest amenity
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

In [5]:
def dist_from_location(house, location):
    """
    this function finds the distance of a location from the 1st address
    First is a dataframe with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    Second is tuple with latitude and longitude of location
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(tqdm(house.iloc[:,0])):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['',100]
        distance = geodesic(flat_loc,location)
        distance = float(str(distance)[:-3]) # convert to float
        flat_amenity[0] = flat
        flat_amenity[1] = distance
        results[flat] = flat_amenity
    return results

House Address

In [6]:
price1999 = pd.read_csv('./data_gov/resale-flat-prices-based-on-approval-date-1990-1999.csv')
price2012 = pd.read_csv('./data_gov/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
price2014 = pd.read_csv('./data_gov/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
price2016 = pd.read_csv('./data_gov/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
price2017 = pd.read_csv('./data_gov/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

df_resale = pd.concat([price1999, price2012, price2014], sort=False)
df_resale = pd.concat([df_resale, price2016, price2017], axis=0, ignore_index=True, sort=False)

In [7]:
df_resale = df_resale[['block', 'street_name']]
df_resale['address'] = df_resale['block'] + ' ' + df_resale['street_name']

In [8]:
text_replace_dict = {"GEORGES":"GEORGE’S",
 "CWEALTH":"COMMONWEALTH",
 "QUEENS":"QUEEN’S",
 "MAMOR":"MA'MOR",
 " ST ":" STREET "
 }
def clean_address(string,dictionary):
 for item in string.split(" "):
# print(item)
    if item in text_replace_dict.keys():
        string = string.replace(item,text_replace_dict[item])
    else:
        string
    return string
 
df_resale['address_clean'] = [clean_address(val,text_replace_dict) for val in df_resale['address']]

In [9]:
df_resale = df_resale.drop(['block','street_name','address'], axis=1)
df_resale.head()

,address_clean
0,309 ANG MO KIO AVE 1
1,309 ANG MO KIO AVE 1
2,309 ANG MO KIO AVE 1
3,309 ANG MO KIO AVE 1
4,216 ANG MO KIO AVE 1


In [10]:
all_address = list(df_resale['address_clean'])
unique_address = list(set(all_address))
print("unique address out of 100255 housing transactions:", len(unique_address))
# unique_address

unique address out of 100255 housing transactions: 9418


In [11]:
unique_address[:10]

['106 WOODLANDS ST 13',
 '213B PUNGGOL WALK',
 '8 EMPRESS RD',
 '636 YISHUN ST 61',
 '258B COMPASSVALE RD',
 '332C ANCHORVALE LINK',
 '632 PASIR RIS DR 3',
 '625 SENJA RD',
 '882 TAMPINES ST 84',
 '117 PENDING RD']

In [12]:
find_postal(unique_address, 'Data/flat_coordinates')

  0%|          | 0/9418 [00:00<?, ?it/s]

In [13]:
flat_coord = pd.read_csv('Data/flat_coordinates.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]
flat_coord.head()

,address,LATITUDE,LONGITUDE
0,106 WOODLANDS ST 13,1.436769,103.782224
1,213B PUNGGOL WALK,1.400720,103.898485
2,8 EMPRESS RD,1.316412,103.805374
3,636 YISHUN ST 61,1.419797,103.839313
4,258B COMPASSVALE RD,1.392903,103.897405


## Get dist from City Hall (town centre)

In [14]:
df_cityhall_coord  = pd.DataFrame([['CITY HALL', 1.29317576, 103.8525073]], columns=['SEARCHVAL', 'LATITUDE', 'LONGITUDE'])
df_cityhall_coord

,SEARCHVAL,LATITUDE,LONGITUDE
0,CITY HALL,1.293176,103.852507


In [69]:
dist_cityhall = dist_from_location(flat_coord, (1.29317576, 103.8525073))
# dist_dhoby = dist_from_location(flat_coord, (1.299308, 103.845285))
dist_cityhall = pd.DataFrame.from_dict(dist_cityhall).T
dist_cityhall = dist_cityhall.rename(columns={0: 'flat', 1: 'dist_cityhall'}).reset_index().drop(['index'], axis=1)
dist_cityhall.head()

  0%|          | 0/11634 [00:00<?, ?it/s]

,flat,dist_cityhall
0,106 WOODLANDS ST 13,17.6999
1,213B PUNGGOL WALK,12.9458
2,8 EMPRESS RD,5.84098
3,636 YISHUN ST 61,14.078
4,258B COMPASSVALE RD,12.1065


In [70]:
dist_cityhall.to_csv('./Data_coord/dist_cityhall.csv')

## Supermarkets

In [17]:
df_supermarket = pd.read_csv('data_gov/list-of-supermarket-licences.csv')
df_supermarket.head()

,licence_num,licensee_name,building_name,block_house_num,level_num,unit_num,street_name,postal_code
0,S02185J000,COLD STORAGE SINGAPORE (1983) PTE LTD,na,982,1,1,BUANGKOK CRESCENT,530982
1,S97166P000,COLD STORAGE SINGAPORE (1983) PTE LTD,HOUGANG GREEN SHOPPING MALL,21,1,52,HOUGANG STREET 51,538719
2,CE15B61K000,U STARS SUPERMARKET PTE. LTD.,na,330,1,1,ANCHORVALE STREET,540330
3,S02029J000,NTUC Fairprice Co-operative Ltd,HOUGANG POINT,1,2,1,HOUGANG STREET 91,538692
4,CE04561V000,SHENG SIONG SUPERMARKET PTE LTD,na,19,1,42401,SERANGOON NORTH AVENUE 5,554913


In [18]:
supermerket_address = list(df_supermarket['postal_code'])
unique_supermarket_address = list(set(supermerket_address))

print('Unique addresses:', len(unique_supermarket_address))

Unique addresses: 445


In [19]:
find_postal(unique_supermarket_address, 'Data/supermarket_coordinates')

  0%|          | 0/445 [00:00<?, ?it/s]

In [20]:
supermarket_coord = pd.read_csv('Data/supermarket_coordinates.csv')
supermarket_coord.drop_duplicates(subset=['address'], inplace=True)
supermarket_coord = supermarket_coord[['SEARCHVAL','LATITUDE','LONGITUDE']]
supermarket_coord.head()

,SEARCHVAL,LATITUDE,LONGITUDE
0,KIZROO KINDERGARTEN,1.338797,103.793256
9,720 CLEMENTI WEST STREET 2 SINGAPORE 120720,1.302464,103.763751
10,PPT LODGE 1B,1.417305,103.896203
13,57 NEW UPPER CHANGI ROAD SINGAPORE 461057,1.324246,103.941569
17,KITS4KIDS SPECIAL SCHOOL,1.366488,103.841853


In [21]:
nearest_supermarket = find_nearest(flat_coord, supermarket_coord)
flat_supermarket = pd.DataFrame.from_dict(nearest_supermarket).T
flat_supermarket = flat_supermarket.rename(columns={0: 'flat', 1: 'supermarket', 2: 'supermarket_dist', 3: 'num_supermarket_2km'}).reset_index().drop(['index','supermarket'], axis=1)
flat_supermarket.head()

  0%|          | 0/11634 [00:00<?, ?it/s]

,flat,supermarket_dist,num_supermarket_2km
0,106 WOODLANDS ST 13,0.523687,8
1,213B PUNGGOL WALK,0.355258,20
2,8 EMPRESS RD,0.968347,15
3,636 YISHUN ST 61,0.541394,16
4,258B COMPASSVALE RD,0.263168,19


In [22]:
flat_supermarket.to_csv('./Data_coord/flat_supermarket.csv')

## Schools

In [23]:
df_primaryschool = pd.read_csv('./data_gov/general-information-of-schools.csv')
df_primaryschool['postal_code'] = df_primaryschool['postal_code'].apply(str)
df_primaryschool.head()

,school_name,address,postal_code
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907
1,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643
2,AI TONG SCHOOL,100 Bright Hill Drive,579646
3,ALEXANDRA PRIMARY SCHOOL,2A Prince Charles Crescent,159016
4,ANCHOR GREEN PRIMARY SCHOOL,31 Anchorvale Drive,544969


In [24]:
school_address = list(df_primaryschool['postal_code'])
unique_school_address = list(set(school_address))
print("no. of schools:", len(unique_school_address))

no. of schools: 183


In [25]:
find_postal(unique_school_address,'Data/school_coordinates')

  0%|          | 0/183 [00:00<?, ?it/s]

In [26]:
school_coord = pd.read_csv('Data/school_coordinates.csv')
school_coord = school_coord[['address','LATITUDE','LONGITUDE']]
school_coord.head()

,address,LATITUDE,LONGITUDE
0,689762,1.381431,103.747154
1,689762,1.381767,103.747290
2,659243,1.354155,103.754210
3,659243,1.354242,103.754166
4,768643,1.433681,103.832924


In [27]:
nearest_school = find_nearest(flat_coord, school_coord)
flat_school = pd.DataFrame.from_dict(nearest_school).T
flat_school = flat_school.rename(columns={0: 'flat', 1: 'school', 2: 'school_dist', 3: 'num_school_2km'}).reset_index().drop('index', axis=1)
flat_school.head()

  0%|          | 0/11634 [00:00<?, ?it/s]

,flat,school,school_dist,num_school_2km
0,106 WOODLANDS ST 13,739067,0.599142,18
1,213B PUNGGOL WALK,828772,0.0876203,40
2,8 EMPRESS RD,268097,0.575657,6
3,636 YISHUN ST 61,769026,0.234271,18
4,258B COMPASSVALE RD,545091,0.176495,44


In [28]:
flat_school.to_csv('./Data_coord/flat_school.csv')

## Hawker and Markets

In [29]:
df_hawker = pd.read_csv('data_gov/list-of-government-markets-hawker-centres.csv')
df_hawker.head(10)

,name_of_centre,location_of_centre,type_of_centre,owner,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls
0,Adam Road Food Centre,"2, Adam Road, S(289876)",HC,Government,32,32,0
1,Amoy Street Food Centre,"National Development Building, Annex B, Telok ...",HC,Government,135,134,1
2,Bedok Food Centre,"1, Bedok Road, S(469572)",HC,Government,32,32,0
3,Beo Crescent Market,"38A, Beo Crescent, S(169982)",MHC,Government,94,32,62
4,Berseh Food Centre,"166, Jalan Besar, S(208877)",HC,Government,66,66,0
5,Bukit Timah Market,"51, Upper Bukit Timah Road, S(588215)",MHC,Government,179,84,95
6,Chomp Chomp Food Centre,"20, Kensington Park Road, S(557269)",HC,Government,36,36,0
7,Commonwealth Crescent Market,"31, Commonwealth Crescent, S(149644)",MHC,Government,102,39,63
8,Dunman Food Centre,"271, Onan Road, S(424768)",HC,Government,30,30,0
9,East Coast Lagoon Food Village,"1220, East Coast Parkway, S(468960)",HC,Government,63,63,0


In [30]:
hawker_name = list(df_hawker['name_of_centre'])
unique_hawker_name = list(set(hawker_name))

print('Unique addresses:', len(unique_hawker_name))

Unique addresses: 107


In [31]:
find_postal(unique_hawker_name, 'Data/hawker_coordinates')

  0%|          | 0/107 [00:00<?, ?it/s]

In [32]:
hawker_coord = pd.read_csv('Data/hawker_coordinates.csv')
hawker_coord = hawker_coord[['address','LATITUDE','LONGITUDE']]
hawker_coord.head()

,address,LATITUDE,LONGITUDE
0,Blk 538 Bedok North Street 3,1.331797,103.925082
1,Newton Food Centre,1.312114,103.839813
2,Newton Food Centre,1.312186,103.839412
3,Newton Food Centre,1.311892,103.839563
4,Blk 117 Aljunied Ave 2,1.320781,103.886774


In [33]:
nearest_hawker = find_nearest(flat_coord, hawker_coord)
flat_hawker = pd.DataFrame.from_dict(nearest_hawker).T
flat_hawker = flat_hawker.rename(columns={0: 'flat', 1: 'hawker', 2: 'hawker_dist', 3: 'num_hawker_2km'}).reset_index().drop('index', axis=1)
flat_hawker.head()

  0%|          | 0/11634 [00:00<?, ?it/s]

,flat,hawker,hawker_dist,num_hawker_2km
0,106 WOODLANDS ST 13,Blks 20/21 Marsiling Lane,0.940258,2
1,213B PUNGGOL WALK,Kovan Market & Food Centre,4.80554,0
2,8 EMPRESS RD,Commonwealth Crescent Market,1.16977,5
3,636 YISHUN ST 61,Chong Pang Market & Food Centre,1.77527,2
4,258B COMPASSVALE RD,Kovan Market & Food Centre,3.94541,0


In [34]:
flat_hawker.to_csv('./Data_coord/flat_hawker.csv')

## Shopping Malls

In [35]:
df_shop = pd.read_csv('./data_other/shoppingmalls.csv')
df_shop.head()

,name
0,100 AM
1,313@Somerset
2,Aperia
3,Balestier Hill Shopping Centre
4,Bugis Cube


In [36]:
shop_name = list(df_shop['name'])
unique_shop_name = list(set(shop_name))

print('Unique addresses:', len(unique_shop_name))

Unique addresses: 176


In [37]:
find_postal(unique_shop_name, 'Data/shoppingmall_coordinates')

  0%|          | 0/176 [00:00<?, ?it/s]

JSONDecodeError


In [38]:
shop_coord = pd.read_csv('Data/shoppingmall_coordinates.csv')
shop_coord.drop_duplicates(subset=['address'], inplace=True)
shop_coord = shop_coord[['address','LATITUDE','LONGITUDE']]
shop_coord.head()

,address,LATITUDE,LONGITUDE
0,Liat Towers,1.305108,103.830728
1,The Centrepoint,1.301978,103.839759
4,Capitol Piazza,1.293063,103.851293
5,Square 2,1.278307,103.850795
10,Palais Renaissance,1.306655,103.829588


In [67]:
nearest_mall = find_nearest(flat_coord, shop_coord)
flat_mall = pd.DataFrame.from_dict(nearest_mall).T
flat_mall = flat_mall.rename(columns={0: 'flat', 1: 'mall', 2: 'mall_dist', 3: 'num_mall_2km'}).reset_index().drop('index', axis=1)
flat_mall.head()

  0%|          | 0/11634 [00:00<?, ?it/s]

,flat,mall,mall_dist,num_mall_2km
0,106 WOODLANDS ST 13,Causeway Point,0.43555,6
1,213B PUNGGOL WALK,Waterway Point,0.753017,6
2,8 EMPRESS RD,The Star Vista,2.15634,0
3,636 YISHUN ST 61,Wisteria Mall,0.292174,3
4,258B COMPASSVALE RD,Compass One,0.276868,7


In [68]:
flat_mall.to_csv('./Data_coord/flat_mall.csv')

## Parks

In [41]:
park_coord = pd.read_csv('data_other/nparks-parks-kml.csv')
park_coord.reset_index(inplace=True)
park_coord = park_coord[['index','Y','X']]
park_coord.head()

,index,Y,X
0,0,1.279077,103.798053
1,1,1.357520,103.867429
2,2,1.311501,103.898950
3,3,1.321547,103.921504
4,4,1.372526,103.829292


In [42]:
nearest_park = find_nearest(flat_coord, park_coord)
flat_park = pd.DataFrame.from_dict(nearest_park).T
flat_park = flat_park.rename(columns={0: 'flat', 1: 'park', 2: 'park_dist', 3: 'num_park_2km'}).reset_index().drop(['index','park'], axis=1)
flat_park.head()

  0%|          | 0/11634 [00:00<?, ?it/s]

,flat,park_dist,num_park_2km
0,106 WOODLANDS ST 13,0.311881,4
1,213B PUNGGOL WALK,1.16783,3
2,8 EMPRESS RD,0.305943,12
3,636 YISHUN ST 61,0.67212,5
4,258B COMPASSVALE RD,0.720714,3


In [43]:
flat_park.to_csv('./Data_coord/flat_park.csv')

## MRT

In [44]:
mrt_df = pd.read_csv('./data_other/mrtsg.csv')
mrt_df.head()

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
3,81,BAKAU LRT STATION,SE3,36026.0821,41113.8766,1.388093,103.905418,OTHERS
4,80,BANGKIT LRT STATION,BP9,21248.2460,40220.9693,1.380018,103.772667,OTHERS


In [45]:
mrt_address = list(mrt_df['STN_NAME'])
unique_mrt_address = list(set(mrt_address))
print("no. of mrt:", len(unique_mrt_address))

no. of mrt: 166


In [46]:
find_postal(unique_mrt_address,'Data/MRT_coordinates')

  0%|          | 0/166 [00:00<?, ?it/s]

In [47]:
mrt_coord = pd.read_csv('./Data/MRT_coordinates.csv')
len(mrt_coord)

841

In [48]:
nan_value = float("NaN")
mrt_coord.replace("", nan_value, inplace=True)
mrt_coord.dropna(subset = ["BLK_NO"], inplace=True)
mrt_coord = mrt_coord[['SEARCHVAL', 'LATITUDE','LONGITUDE']]
mrt_coord.head()

,SEARCHVAL,LATITUDE,LONGITUDE
2,JOO KOON MRT STATION (EW29),1.327717,103.678375
3,UOB JOO KOON MRT STATION,1.327689,103.678240
4,DBS JOO KOON MRT STATION,1.327694,103.678085
5,OCBC JOO KOON MRT STATION,1.327686,103.678346
8,MATTAR MRT STATION (DT25),1.326877,103.883248


In [49]:
nearest_mrt = find_nearest(flat_coord, mrt_coord)
flat_mrt = pd.DataFrame.from_dict(nearest_mrt).T
flat_mrt = flat_mrt.rename(columns={0: 'flat', 1: 'mrt', 2: 'mrt_dist', 3: 'num_mrt_2km'}).reset_index().drop('index', axis=1)
flat_mrt.head()

  0%|          | 0/11634 [00:00<?, ?it/s]

,flat,mrt,mrt_dist,num_mrt_2km
0,106 WOODLANDS ST 13,UOB WOODLANDS MRT STATION,0.493527,9
1,213B PUNGGOL WALK,SOO TECK LRT STATION (PW7),0.526115,31
2,8 EMPRESS RD,FARRER ROAD MRT STATION (CC20),0.250424,21
3,636 YISHUN ST 61,UOB KHATIB MRT STATION,0.737341,9
4,258B COMPASSVALE RD,OCBC SENGKANG MRT STATION,0.246773,31


In [50]:
flat_mrt.to_csv('./Data_coord/flat_mrt.csv')

## NPC

In [51]:
crime = pd.read_csv('./data_gov/five-preventable-crime-cases-recorded-by-neighbourhood-police-centre-npc-annual.csv')

In [52]:
newcol = crime['level_1'].str.split(" - ", expand = True)
crime["npc"]= newcol[1]
crime['year'] = crime['year'].astype(int)
crime1 = crime[(crime['year'] >= 2017) & (crime['npc'] != 'Total') & (crime['value'] != 'na') & (crime['value'] != '-')]
crime1['value'] = crime1['value'].astype(float)
crime2 = pd.DataFrame(crime1).reset_index()
crime2.drop(['index','level_1'], axis=1,inplace=True)
crime3 = crime2[['npc','year','value']]
crime3.head(5)

<ipython-input-52-9477880324bb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime1['value'] = crime1['value'].astype(float)


,npc,year,value
0,Bukit Merah East NPC,2017,89.0
1,Marina Bay NPC,2017,134.0
2,Rochor NPC,2017,89.0
3,Bukit Merah West NPC,2017,64.0
4,Clementi NPC,2017,50.0


In [53]:
npc_address = list(crime3['npc'])
unique_npc_address = list(set(npc_address))
print("no. of npc:", len(unique_npc_address))

no. of npc: 35


In [54]:
find_postal(unique_npc_address,'Data/npc_coordinates')

  0%|          | 0/35 [00:00<?, ?it/s]

In [55]:
npc_coord = pd.read_csv('./Data/npc_coordinates.csv')
npc_coord.drop_duplicates(subset=['address'], inplace=True)
npc_coord = npc_coord[['SEARCHVAL','LATITUDE','LONGITUDE']]
npc_coord.head()

,SEARCHVAL,LATITUDE,LONGITUDE
0,SERANGOON NEIGHBOURHOOD POLICE CENTRES,1.352018,103.870533
2,YISHUN NORTH NEIGHBOURHOOD POLICE CENTRE,1.429343,103.840294
3,ROCHOR NEIGHBOURHOOD POLICE CENTRE,1.307619,103.854219
4,MARINA BAY NEIGHBOURHOOD POLICE CENTRE,1.274735,103.848994
5,PUNGGOL NEIGHBOURHOOD POLICE CENTRE,1.394196,103.916943


In [65]:
nearest_npc = find_nearest(flat_coord, npc_coord)
flat_npc = pd.DataFrame.from_dict(nearest_npc).T
flat_npc = flat_npc.rename(columns={0: 'flat', 1: 'npc', 2: 'npc_dist', 3: 'num_npc_2km'}).reset_index().drop('index', axis=1)
flat_npc.head()

  0%|          | 0/11634 [00:00<?, ?it/s]

,flat,npc,npc_dist,num_npc_2km
0,106 WOODLANDS ST 13,WOODLANDS WEST NEIGHBOURHOOD POLICE CENTRE,0.520801,1
1,213B PUNGGOL WALK,SENGKANG NEIGHBOURHOOD POLICE CENTRE,1.01454,1
2,8 EMPRESS RD,BUKIT TIMAH NEIGHBOURHOOD POLICE CENTRE,1.04703,1
3,636 YISHUN ST 61,YISHUN SOUTH NEIGHBOURHOOD POLICE CENTRE,0.696971,2
4,258B COMPASSVALE RD,SENGKANG NEIGHBOURHOOD POLICE CENTRE,0.389549,1


In [66]:
flat_npc.to_csv('./Data_coord/flat_npc.csv')

## Merge All

In [71]:
# flat_school = pd.read_csv('./Data_coord/flat_school.csv')
# flat_hawker = pd.read_csv('./Data_coord/flat_hawker.csv')
# flat_park = pd.read_csv('./Data_coord/flat_park.csv')
# flat_mall = pd.read_csv('./Data_coord/flat_mall.csv')
# flat_mrt = pd.read_csv('./Data_coord/flat_mrt.csv')
# flat_supermarket = pd.read_csv('./Data_coord/flat_supermarket.csv')
# flat_npc = pd.read_csv('./Data_coord/flat_npc.csv')
## Merge all

flat_amenities = flat_school.merge(flat_hawker, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_park, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_mall, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_mrt, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_supermarket, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_npc, on='flat', how='outer')

flat_amenities.head()

,flat,school,school_dist,num_school_2km,hawker,hawker_dist,num_hawker_2km,park_dist,num_park_2km,mall,mall_dist,num_mall_2km,mrt,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km,npc,npc_dist,num_npc_2km
0,106 WOODLANDS ST 13,739067,0.599142,18,Blks 20/21 Marsiling Lane,0.940258,2,0.311881,4,Causeway Point,0.43555,6,UOB WOODLANDS MRT STATION,0.493527,9,0.523687,8,WOODLANDS WEST NEIGHBOURHOOD POLICE CENTRE,0.520801,1
1,213B PUNGGOL WALK,828772,0.0876203,40,Kovan Market & Food Centre,4.80554,0,1.16783,3,Waterway Point,0.753017,6,SOO TECK LRT STATION (PW7),0.526115,31,0.355258,20,SENGKANG NEIGHBOURHOOD POLICE CENTRE,1.01454,1
2,8 EMPRESS RD,268097,0.575657,6,Commonwealth Crescent Market,1.16977,5,0.305943,12,The Star Vista,2.15634,0,FARRER ROAD MRT STATION (CC20),0.250424,21,0.968347,15,BUKIT TIMAH NEIGHBOURHOOD POLICE CENTRE,1.04703,1
3,636 YISHUN ST 61,769026,0.234271,18,Chong Pang Market & Food Centre,1.77527,2,0.67212,5,Wisteria Mall,0.292174,3,UOB KHATIB MRT STATION,0.737341,9,0.541394,16,YISHUN SOUTH NEIGHBOURHOOD POLICE CENTRE,0.696971,2
4,258B COMPASSVALE RD,545091,0.176495,44,Kovan Market & Food Centre,3.94541,0,0.720714,3,Compass One,0.276868,7,OCBC SENGKANG MRT STATION,0.246773,31,0.263168,19,SENGKANG NEIGHBOURHOOD POLICE CENTRE,0.389549,1


In [72]:
flat_amenities.to_csv('Data/flat_amenities.csv', index=False)

In [73]:
flat_amenities = pd.read_csv('Data/flat_amenities.csv')
flat_amenities = flat_amenities.merge(dist_cityhall, on='flat', how='outer')
flat_amenities.head()

,flat,school,school_dist,num_school_2km,hawker,hawker_dist,num_hawker_2km,park_dist,num_park_2km,mall,...,num_mall_2km,mrt,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km,npc,npc_dist,num_npc_2km,dist_cityhall
0,106 WOODLANDS ST 13,739067,0.599142,18,Blks 20/21 Marsiling Lane,0.940258,2,0.311881,4,Causeway Point,...,6,UOB WOODLANDS MRT STATION,0.493527,9,0.523687,8,WOODLANDS WEST NEIGHBOURHOOD POLICE CENTRE,0.520801,1,17.6999
1,213B PUNGGOL WALK,828772,0.087620,40,Kovan Market & Food Centre,4.805537,0,1.167833,3,Waterway Point,...,6,SOO TECK LRT STATION (PW7),0.526115,31,0.355258,20,SENGKANG NEIGHBOURHOOD POLICE CENTRE,1.014541,1,12.9458
2,8 EMPRESS RD,268097,0.575657,6,Commonwealth Crescent Market,1.169769,5,0.305943,12,The Star Vista,...,0,FARRER ROAD MRT STATION (CC20),0.250424,21,0.968347,15,BUKIT TIMAH NEIGHBOURHOOD POLICE CENTRE,1.047034,1,5.84098
3,636 YISHUN ST 61,769026,0.234271,18,Chong Pang Market & Food Centre,1.775271,2,0.672120,5,Wisteria Mall,...,3,UOB KHATIB MRT STATION,0.737341,9,0.541394,16,YISHUN SOUTH NEIGHBOURHOOD POLICE CENTRE,0.696971,2,14.078
4,258B COMPASSVALE RD,545091,0.176495,44,Kovan Market & Food Centre,3.945409,0,0.720714,3,Compass One,...,7,OCBC SENGKANG MRT STATION,0.246773,31,0.263168,19,SENGKANG NEIGHBOURHOOD POLICE CENTRE,0.389549,1,12.1065


In [74]:
flat_amenities.to_csv('Data/flat_amenities.csv', index=False)